In [1]:
import numpy as np
import pandas as pd
import re

from collections import Counter
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RepeatedKFold

%matplotlib inline
# %pylab inline

In [16]:
train = pd.read_csv('dataset/train.csv')
test = pd.read_csv('dataset/test.csv')

In [17]:
def transf(valor):
        if valor == 'female':
            return 1
        else:
            return 0

train['Sex'] = train['Sex'].map(transf)
test['Sex'] = test['Sex'].map(transf)

In [18]:
media = np.int(train['Age'].mean())
train['Age'] = train['Age'].fillna(media)
test['Age'] = test['Age'].fillna(media)

In [19]:
train['Ticket'] = train['Ticket'].str.replace(r'\D', '')
train['Ticket'] = pd.to_numeric(train['Ticket'])

test['Ticket'] = test['Ticket'].str.replace(r'\D', '')
test['Ticket'] = pd.to_numeric(test['Ticket'])

In [20]:
vari = ['Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Pclass']
x = train[vari]
y = train['Survived']
x = x.fillna(-1)
y = y.fillna(-1)

In [21]:
lista = train['Ticket']

contador = Counter(lista)

repetidos = [
    item for item, quantidade in contador.items() 
        if quantidade > 1
]

quantidade_repetidos = len(repetidos)

print(f'Há {quantidade_repetidos} números repetidos na lista')

Há 134 números repetidos na lista


In [30]:
# alterando quantidade de filhos
idx = train[train['Age'] < 15].index
train['Parch'][idx] = 0
test['Parch'][idx] = 0

/home/rafael/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/rafael/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


ValueError: [419 435 445 446 448 469 479 480 489 530 535 541 542 549 618 634 642 644
 683 686 691 720 731 750 751 755 777 780 787 788 802 803 813 819 824 827
 831 850 852 869] not contained in the index

In [23]:
df1 = train.set_index('Ticket')
d = df1[['Survived', 'Pclass', 'SibSp', 'Parch', 'Sex','Age', 'Name']].loc[repetidos]
d = d.rename(columns = {'SibSp':'Numero irmaos', 'Parch': 'Filhos a bordo'})
d.loc[347742.0]

,Survived,Pclass,Numero irmaos,Filhos a bordo,Sex,Age,Name
Ticket,,,,,,,
347742.0,1,3,0,2,1,27.0,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)"
347742.0,1,3,1,0,1,1.0,"Johnson, Miss. Eleanor Ileen"
347742.0,1,3,1,0,0,4.0,"Johnson, Master. Harold Theodor"


In [24]:
resul = []

kf = RepeatedKFold(n_splits=2, n_repeats=10, random_state=10)

for linhas_treino, linhas_test in kf.split(x):

    x_treino, x_valid = x.iloc[linhas_treino], x.iloc[linhas_test]
    y_treino, y_valid = y.iloc[linhas_treino], y.iloc[linhas_test]

    modelo = DecisionTreeClassifier(max_depth=2)
    modelo.fit(x_treino, y_treino)

    p = modelo.predict(x_valid)

    acc = np.mean(y_valid == p)
    
    resul.append(acc)
    print("Acuracia: ", acc)

Acuracia:  0.8004484304932735
Acuracia:  0.7573033707865169
Acuracia:  0.7937219730941704
Acuracia:  0.7662921348314606
Acuracia:  0.7802690582959642
Acuracia:  0.7730337078651686
Acuracia:  0.7892376681614349
Acuracia:  0.7640449438202247
Acuracia:  0.7847533632286996
Acuracia:  0.7842696629213484
Acuracia:  0.7802690582959642
Acuracia:  0.7887640449438202
Acuracia:  0.7959641255605381
Acuracia:  0.7640449438202247
Acuracia:  0.7802690582959642
Acuracia:  0.7820224719101123
Acuracia:  0.7937219730941704
Acuracia:  0.7752808988764045
Acuracia:  0.7825112107623319
Acuracia:  0.7887640449438202


In [25]:
np.mean(resul)

0.7812493072000806

In [26]:
x_valid_check = train.iloc[linhas_test].copy()
x_valid_check['p'] = p
x_valid_check[['Survived', 'p']].head()

,Survived,p
0,0,0
1,1,1
2,1,0
3,1,1
4,0,0


In [27]:
modelo.fit(x,y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=2,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [28]:
p_class = modelo.predict(test[vari].fillna(-1))

In [29]:
tree_sub = pd.Series(p_class, index= test['PassengerId'], name='Survived')
tree_sub.to_csv("modelo_dectree2.csv", header=True)